In [7]:
"""G=(VN, VT, P, S,), VN ={S, A, B, C, D}, VT ={a,b,c,d,e},
P={ 
1. S → B 
2. B → C 
3. B → C c B
4. C → a d D 
5. D → A e 
6. A → b 
7. A → A b, 
8. B → d 
}"""

'G=(VN, VT, P, S,), VN ={S, A, B, C, D}, VT ={a,b,c,d,e},\nP={ \n1. S → B \n2. B → C \n3. B → C c B\n4. C → a d D \n5. D → A e \n6. A → b \n7. A → A b, \n8. B → d \n}'

In [8]:
Prod = {'S':['B'], 'A':['b', 'Ab'], 'B':['C', 'CcB', 'd'], 'C':['sdD'], 'D':['A'] }

In [51]:
first={}
def frst(Prod):
    for state, rule in Prod.items():
        f_list = []
        for s in rule:
            if s[0] not in f_list:
                f_list.append(s[0])
        first[state] = f_list
    for k, v in first.items():
        found = []
        for st in v:
            if st[0] in first.keys():
                found.extend(first[st[0]])
        for s in found:
            if s not in v:
                v.extend(s)
    for nk, nv in first.items():
        print(f'first{nk} | {nv}')
frst(Prod)

firstS | ['B', 'C', 'd']
firstA | ['b', 'A']
firstB | ['C', 'd', 's']
firstC | ['s']
firstD | ['A', 'b']


In [52]:
# Prod = {'S':['B'], 'A':['b', 'Ab'], 'B':['C', 'CcB', 'd'], 'C':['sdD'], 'D':['A'] }
last={}
def lst(Prod):
    for lstate, lrule in Prod.items():
        l_list = []
        for ns in lrule:
            if ns[-1] not in l_list:
                l_list.append(ns[-1])
        last[lstate] = l_list

    for lk, lv in last.items():
        lfound = []
        for lst in v:
            if lst[-1] in last.keys():
                lfound.extend(last[lst[-1]])
        for ls in lfound:
            if ls not in lv:
                lv.extend(ls)
    for lk, lv in last.items():
        print(f'last{lk} | {lv}')
lst(Prod)

lastS | ['B', 'b']
lastA | ['b']
lastB | ['C', 'B', 'd', 'b']
lastC | ['D', 'b']
lastD | ['A', 'b']


In [54]:
# Prod = {'S':['B'], 'A':['b', 'Ab'], 'B':['C', 'CcB', 'd'], 'C':['sdD'], 'D':['A'] }
print(Prod)
precedence_matrix = {}
def prcd_mtrx(Prod):
    for r in Prod.values():
        for rule_s in r:
            if len(rule_s)>1:
                for i in range(0,len(rule_s)-1):   
                    #neighbors are = in precedence                
                    precedence_matrix[rule_s[i][0]] = [['=', rule_s[(i+1)]],['-'],['-']]    # symbol : ['=', smthg] ['<', smthg] ['>', smthg]   |  '-' for none
                    if rule_s[i][0] in first.keys():
                        for r in first[rule_s[i][0]]:
                            #but left-side neighbor < its first(right-side neighbor)
                            if r not in precedence_matrix.keys():
                                precedence_matrix[str(r)] = [['-'],['<', rule_s[(i+1)]],['-']]   
                            elif r in precedence_matrix.keys():
                                precedence_matrix[str(r)][1] = ['<', rule_s[(i+1)]]
                    if rule_s[i][0] in last.keys() and rule_s[i+1][0] in first.keys():
                        for rl in last[rule_s[i][0]]:
                            #but last(left-side neighbor) > its first(right-side neighbor)
                            if rl not in precedence_matrix.keys():
                                for f in first(rule_s[(i+1)]):
                                    precedence_matrix[str(rl)] = [['-'],['-'],['>', f]]   
                            elif r in precedence_matrix.keys():
                                precedence_matrix[str(r)][2] = ['>', f]
                    if rule_s[i][0] in last.keys() and rule_s[i+1][0] not in first.keys():
                    #but last(left-side neighbor) > its right-side terminal-symbol neighbor
                        for rl in last[rule_s[i][0]]:
                            if rl not in precedence_matrix.keys():
                                precedence_matrix[str(rl)] = [['-'],['-'],['>', rule_s[i+1]]]   
                            elif r in precedence_matrix.keys() and len(rule_s)>1:
                                precedence_matrix[str(r)][2] = ['>', rule_s[i+1]]
prcd_mtrx(Prod)
precedence_matrix

{'S': ['B'], 'A': ['b', 'Ab'], 'B': ['C', 'CcB', 'd'], 'C': ['sdD'], 'D': ['A']}


{'A': [['=', 'b'], ['<', 'b'], ['>', 'b']],
 'b': [['-'], ['<', 'b'], ['-']],
 'C': [['=', 'c'], ['-'], ['-']],
 's': [['=', 'd'], ['-'], ['-']],
 'D': [['-'], ['-'], ['>', 'c']],
 'c': [['=', 'B'], ['-'], ['-']],
 'd': [['=', 'D'], ['-'], ['-']]}

In [56]:
strin = 'adabcd'
strin2 = 'sdcsdb'  #This one is accepted
def pars(stri):
    stack = ['$']
    accepted = False
    for char in stri:
        stack.append(char)
    stack.append('$')
    i_cr = 1
    while i_cr <(len(stack)-1):
        print(f'Analyzing: {stack[i_cr]}')
        if stack[i_cr] in precedence_matrix.keys():
            accepted = True
            current = stack[i_cr]
            nex = stack[i_cr+1]
            if nex in precedence_matrix[current][0] or nex in precedence_matrix[current][1]:  #current<=next
                stack.remove(current)
                current = nex
                i_cr+=1
            elif nex in precedence_matrix[current][2]:  #current>next
                stack.remove(current)
                i_cr +=1
        else:
            accepted = False
            break
        i_cr +=1
    if accepted == True:
        print('Your input is accepted.')
    else:
        print(f'This is what\'s left in stack:\n{stack}')
        print('Your input is not accepted.')
pars(strin)

Analyzing: a
This is what's left in stack:
['$', 'a', 'd', 'a', 'b', 'c', 'd', '$']
Your input is not accepted.


In [64]:
print(Prod, '\n')
frst(Prod)
print('\n')
lst(Prod)
print('\n')
prcd_mtrx(Prod)
for ch, vl in precedence_matrix.items():
    print(ch,' -  ', vl, '\n')
pars(strin)

{'S': ['B'], 'A': ['b', 'Ab'], 'B': ['C', 'CcB', 'd'], 'C': ['sdD'], 'D': ['A']} 

firstS | ['B', 'C', 'd']
firstA | ['b', 'A']
firstB | ['C', 'd', 's']
firstC | ['s']
firstD | ['A', 'b']


lastS | ['B', 'b']
lastA | ['b']
lastB | ['C', 'B', 'd', 'b']
lastC | ['D', 'b']
lastD | ['A', 'b']


A  -   [['=', 'b'], ['<', 'b'], ['>', 'b']] 

b  -   [['-'], ['<', 'b'], ['-']] 

C  -   [['=', 'c'], ['-'], ['-']] 

s  -   [['=', 'd'], ['-'], ['-']] 

D  -   [['-'], ['-'], ['>', 'c']] 

c  -   [['=', 'B'], ['-'], ['-']] 

d  -   [['=', 'D'], ['-'], ['-']] 

Analyzing: a
This is what's left in stack:
['$', 'a', 'd', 'a', 'b', 'c', 'd', '$']
Your input is not accepted.
